In [17]:
import pandas as pd
import json
import os
import dotenv
from pymongo import MongoClient


In [18]:
# I create the connection with Compass, because I have the data already there.
client = MongoClient("localhost:27017")
db = client['geospatial']
c = db.get_collection('companies')


In [19]:
# I look for all those companies that have offices in Australia
projection = {}
filter_ = {'offices.country_code':'AUS'}


In [20]:
result = list(c.find(filter_,projection))
len(result)
print(f'I got {len(result)} companies in AUS')
result

I got 257 companies in AUS


[{'_id': ObjectId('52cdef7c4bab8bd675297dba'),
  'name': 'Google',
  'permalink': 'google',
  'crunchbase_url': 'http://www.crunchbase.com/company/google',
  'homepage_url': 'http://google.com',
  'blog_url': 'http://googleblog.blogspot.com',
  'blog_feed_url': 'http://googleblog.blogspot.com/feeds/posts/default?alt=rss',
  'twitter_username': 'google',
  'category_code': 'search',
  'number_of_employees': 28000,
  'founded_year': 1998,
  'founded_month': 9,
  'founded_day': 7,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'search, gmail, blogger, find, information, youtube, google',
  'alias_list': 'Google',
  'email_address': 'google@google.com',
  'phone_number': '650.253.0000',
  'description': '',
  'created_at': 'Sat Jun 30 22:18:12 UTC 2007',
  'updated_at': 'Mon Jan 06 04:52:54 UTC 2014',
  'overview': '<p>Google provides search and advertising services, which together aim to organize and monetize the wo

In [21]:
dict_of_companies = {}
for i in result:
    dict_of_companies[i['name']]= i['offices']


In [26]:
# I create a dictionary with information about the name of the company, the type, and its office. 
# Taking only those offices that are in Australia. 
offices_aus = []
for company in dict_of_companies.keys():
    list_of_ofices = dict_of_companies[company]
    for office in list_of_ofices:
        if office['country_code'] == 'AUS':
            offices_aus.append({'company':company,
                                'name_office':office['description'],
                                'city': office['city'],
                                'address': office['address1'],
                                'location':{'type':'Point',
                                            'coordinates':[office['latitude'],office['longitude']]}})

In [27]:
offices_aus

[{'commpany': 'Google',
  'name_office': 'Google Melbourne',
  'city': 'Melbourne',
  'address': 'Rialto South Tower, Level 27',
  'location': {'type': 'Point', 'coordinates': [-37.879234, 145.073608]}},
 {'commpany': 'Google',
  'name_office': 'Google Sydney',
  'city': 'Sydney',
  'address': 'Levels 11 & 18, Tower 1, Darling Park',
  'location': {'type': 'Point', 'coordinates': [-34.822723, 138.612396]}},
 {'commpany': 'Truphone',
  'name_office': 'Tru Australia',
  'city': 'Sydney',
  'address': 'Level 33, Australia Square',
  'location': {'type': 'Point', 'coordinates': [None, None]}},
 {'commpany': 'Boonex',
  'name_office': None,
  'city': 'Sydney',
  'address': '10 Cedar Grove',
  'location': {'type': 'Point', 'coordinates': [-33.722577, 150.981801]}},
 {'commpany': 'TokBox',
  'name_office': 'Sydney Office',
  'city': 'Ultimo',
  'address': 'Fishburners',
  'location': {'type': 'Point', 'coordinates': [None, None]}},
 {'commpany': 'KIT digital',
  'name_office': 'Melbourne Offi

In [28]:
df_aus_companies = pd.DataFrame(offices_aus)

In [29]:
df_aus_companies

,commpany,name_office,city,address,location
0,Google,Google Melbourne,Melbourne,"Rialto South Tower, Level 27","{'type': 'Point', 'coordinates': [-37.879234, ..."
1,Google,Google Sydney,Sydney,"Levels 11 & 18, Tower 1, Darling Park","{'type': 'Point', 'coordinates': [-34.822723, ..."
2,Truphone,Tru Australia,Sydney,"Level 33, Australia Square","{'type': 'Point', 'coordinates': [None, None]}"
3,Boonex,None,Sydney,10 Cedar Grove,"{'type': 'Point', 'coordinates': [-33.722577, ..."
4,TokBox,Sydney Office,Ultimo,Fishburners,"{'type': 'Point', 'coordinates': [None, None]}"
...,...,...,...,...,...
256,CommTel Network Solutions,Head Office,Tullamarine,46 Ovata Drive,"{'type': 'Point', 'coordinates': [-37.709071, ..."
257,CommTel Network Solutions,Administration & Sales,Tullamarine,8 Mohr Street,"{'type': 'Point', 'coordinates': [-37.711099, ..."
258,CommTel Network Solutions,Western Region Office,Victoria Park,P.O. Box 4361,"{'type': 'Point', 'coordinates': [-37.034419, ..."
259,Novera,,Sydney,Level 4,"{'type': 'Point', 'coordinates': [-33.868898, ..."
